<font color='blue'> **PEST_MF2005** </font>  

In [1]:
import os ; import sys ; import numpy as np ; import flopy ; from pathlib import Path
print(sys.version) ; print(f"numpy version: {np.__version__}") ; print(f"flopy version: {flopy.__version__}")

3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
numpy version: 1.26.4
flopy version: 3.9.1


In [2]:
sim_ws    = Path("./07_PEST")                            ; sim_ws.mkdir(exist_ok=True)     ; modelname = "PEST"

In [3]:
nlay = 3 ; nrow = 20 ; ncol = 20
mm   = flopy.modflow.Modflow   (modelname=modelname, model_ws=sim_ws)
dis = flopy.modflow.ModflowDis (mm, nlay, nrow, ncol)
lpf = flopy.modflow.ModflowLpf (mm, hk=10.0)

In [4]:
mfpackage = "lpf" ; partype = "hk" ; parname = "HK_LAYER_1"
idx  = np.empty((nlay, nrow, ncol), dtype=bool)               ; idx[0] = True   ; idx[1:] = False       ; span = {"idx": idx}
startvalue = 10.0 ; lbound = 0.001 ; ubound = 1000.0          ; transform = "log"
pp = flopy.pest.Params(mfpackage, partype, parname, startvalue, lbound, ubound, span)

mfpackage = "lpf" ; partype = "hk" ; parname = "HK_LAYER_1-3"                                            ; span = {"layers": [0, 2]}
startvalue = 10.0 ; lbound = 0.001 ; ubound = 1000.0 ; transform = "log"
pp = flopy.pest.Params (mfpackage, partype, parname, startvalue, lbound, ubound, span)
tw = flopy.pest.templatewriter.TemplateWriter(mm, [pp])
tw.write_template()

The following packages will be parameterized: ['LPF']



**Multi**

In [5]:
zonearray = np.ones((nlay, nrow, ncol), dtype=int)  ; zonearray[0, 10:, 7:] = 2 ; zonearray[0, 15:, 9:] = 3 ; zonearray[1] = 4

In [6]:
mfpackage = "lpf" ; parzones = [2, 3, 4] ; parvals = [56.777, 78.999, 99.0] ; lbound = 5 ; ubound = 500 ; transform = "log"
plisthk = flopy.pest.zonearray2params(mfpackage, "hk", parzones, lbound, ubound, parvals, transform, zonearray)

In [7]:
parzones = [1, 2] ; parvals = [0.001, 0.0005] ; zonearray = np.ones((nlay, nrow, ncol), dtype=int) ; zonearray[1] = 2
plistvk = flopy.pest.zonearray2params(mfpackage, "vka", parzones, lbound, ubound, parvals, transform, zonearray)

In [8]:
plist = plisthk + plistvk
for p in plist:  print(p.name, p.mfpackage, p.startvalue)

hk_2 lpf 56.777
hk_3 lpf 78.999
hk_4 lpf 99.0
vka_1 lpf 0.001
vka_2 lpf 0.0005


In [9]:
tw = flopy.pest.templatewriter.TemplateWriter(mm, plist)   ; tw.write_template()

The following packages will be parameterized: ['LPF']



**Two-Dimensional Transient Arrays** 'kper' key (list of stress periods) 'idx' key ( multiplier | iarray and jarray) 

In [10]:
nlay = 3 ; nrow = 5 ; ncol = 5 ; nper = 3
mm = flopy.modflow.Modflow(modelname=modelname, model_ws=sim_ws)
dis = flopy.modflow.ModflowDis(mm, nlay, nrow, ncol, nper=nper)
lpf = flopy.modflow.ModflowLpf(mm, hk=10.0)
rch = flopy.modflow.ModflowRch(mm, rech={0: 0.001, 2: 0.003})

In [11]:
plist = [] ;mfpackage = "rch" ;partype = "rech" ;parname = "RECH_MULT" ;startvalue=None ;lbound=None ;ubound=None ;transform=None ;idx=None
span = {"kpers": [0, 1, 2], "idx": idx}   ; pp = flopy.pest.Params(mfpackage, partype, parname, startvalue, lbound, ubound, span) ; plist.append(pp)
tw = flopy.pest.TemplateWriter(mm, plist) ; tw.write_template()

The following packages will be parameterized: ['RCH']



In [12]:
plist = [] ;mfpackage = "rch" ;partype = "rech" ;parname = "RECH_MULT" ;startvalue=None ;lbound=None ;ubound=None ;transform=None
span = {"kpers": [1, 2], "idx": None}     ; pp = flopy.pest.Params(mfpackage, partype, parname, startvalue, lbound, ubound, span) ; plist.append(pp)

In [13]:
plist = [] ;mfpackage = "rch" ;partype = "rech" ;parname = "RECH_MULT" ;startvalue=None ;lbound=None ;ubound=None ;transform=None
idx = np.empty((nrow, ncol), dtype=bool)  ; idx[0:3, 0:3] = True  ; span = {"kpers": [1], "idx": idx}
pp = flopy.pest.Params(mfpackage, partype, parname, startvalue, lbound, ubound, span)  ; plist.append(pp)

In [14]:
# Write the template file
tw = flopy.pest.templatewriter.TemplateWriter(mm, plist)
tw.write_template()

# Print the results
lines = open(os.path.join(sim_ws, "PEST.rch.tpl")).readlines()
for l in lines:
    print(l.strip())

The following packages will be parameterized: ['RCH']

ptf ~
# RCH package for MODFLOW-2005 generated by Flopy 3.9.1
3         0
1        -1 # Stress period 1
CONSTANT    1.000000E-03                           #rech_1
1        -1 # Stress period 2
INTERNAL 1.0 (FREE) -1      #rech_1
~  RECH_MULT  ~ ~  RECH_MULT  ~ ~  RECH_MULT  ~           0.001           0.001
~  RECH_MULT  ~ ~  RECH_MULT  ~ ~  RECH_MULT  ~           0.001           0.001
~  RECH_MULT  ~ ~  RECH_MULT  ~ ~  RECH_MULT  ~           0.001           0.001
0.001           0.001           0.001           0.001           0.001
0.001           0.001           0.001           0.001           0.001
1        -1 # Stress period 3
CONSTANT    3.000000E-03                           #rech_3
